In [22]:
cd /home/code/code/DNN-based_source_separation-main/egs/tutorials/common

/home/code/code/DNN-based_source_separation-main/egs/tutorials/common


In [4]:
"""
!git clone https://github.com/tky1117/DNN-based_source_separation.git
!pip install soundfile
"""

/bin/bash: git: command not found


In [ ]:
!python3 ./local/pad_audio3.py \
--input_dir "/home/code/code/DNN-based_source_separation-main/dataset/all" \
--train_dir "/home/code/code/DNN-based_source_separation-main/dataset/all_train" \
--test_dir "/home/code/code/DNN-based_source_separation-main/dataset/all_test" \
--sample_rate 22000 \
--n_patterns 10 \
--seed 42


In [23]:
!python3 ./local/dataset_mine4.py \
--train_root "/home/code/code/DNN-based_source_separation-main/dataset/all_train_pad_10"  \
--test_root "/home/code/code/DNN-based_source_separation-main/dataset/all_test_pad_10" \
--all_n_root "/home/code/code/DNN-based_source_separation-main/dataset/all_10_2" \
--sample_rate 16000 --n_sources 2

Finding global max: 100%|███████████████████| 4120/4120 [01:54<00:00, 35.97it/s]
/home/code/code/DNN-based_source_separation-main/dataset/all_train_pad_10
['2023-06-27 13-33-55_105623492.00-105880119.00_padded_0.wav', '2023-06-27 13-33-55_105623492.00-105880119.00_padded_1.wav', '2023-06-27 13-33-55_105623492.00-105880119.00_padded_2.wav', '2023-06-27 13-33-55_105623492.00-105880119.00_padded_3.wav', '2023-06-27 13-33-55_105623492.00-105880119.00_padded_4.wav', '2023-06-27 13-33-55_105623492.00-105880119.00_padded_5.wav', '2023-06-27 13-33-55_105623492.00-105880119.00_padded_6.wav', '2023-06-27 13-33-55_105623492.00-105880119.00_padded_7.wav', '2023-06-27 13-33-55_105623492.00-105880119.00_padded_8.wav', '2023-06-27 13-33-55_105623492.00-105880119.00_padded_9.wav', '2023-06-27 13-33-55_118144513.00-118575771.00_padded_0.wav', '2023-06-27 13-33-55_118144513.00-118575771.00_padded_1.wav', '2023-06-27 13-33-55_118144513.00-118575771.00_padded_2.wav', '2023-06-27 13-33-55_118144513.00-1185

In [ ]:
import os
import json

def check_and_clean_data(file_path):
    # Load JSON data
    with open(file_path, 'r') as file:
        data = json.load(file)

    # List to hold indices of items to be removed
    indices_to_remove = []

    # Check each item in the data
    for index, item in enumerate(data):
        sources = item.get('sources', {})
        for source_key, source_info in sources.items():
            path = source_info.get('path')
            if not os.path.exists(path):
                indices_to_remove.append(index)
                break

    # Remove items with missing files
    for index in sorted(indices_to_remove, reverse=True):
        del data[index]

    # Save the cleaned data back to file
    with open(file_path, 'w') as file:
        json.dump(data, file, indent=4)

# Paths to your train and test JSON files
train_file_path = '/content/drive/MyDrive/DNN-based_source_separation/dataset/all_10/train.json'
test_file_path = '/content/drive/MyDrive/DNN-based_source_separation/dataset/all_10/test.json'

# Cleaning the train and test datasets
check_and_clean_data(train_file_path)
check_and_clean_data(test_file_path)

print("Data cleaning complete.")


In [ ]:
import json
import re

def load_json_file(file_path):
    """
    JSONファイルを読み込み、内容を返す関数。
    """
    with open(file_path, 'r', encoding='utf-8') as file:
        return json.load(file)

def extract_and_process_paths(data):
    """
    JSONデータからpathの値を抽出し、必要な文字列を削除する。
    """
    paths = []
    for item in data:
        for source in item["sources"].values():
            path = source["path"]
            # 正規表現で 'normalized_' と 'padded_0' から 'padded_10' を削除
            processed_path = re.sub(r'normalized_|padded_[0-9]|padded_10', '', path)
            paths.append(processed_path)
    return paths

def find_duplicates(train_paths, test_paths):
    """
    test.jsonのパスがtrain.jsonに含まれているかどうかを確認する。
    """
    train_set = set(train_paths)
    duplicates = [path for path in test_paths if path in train_set]
    return duplicates

# JSONファイルのパス
train_json_path = '../../../../DNN-based_source_separation/dataset/all_10_2/train.json'
test_json_path = '../../../../DNN-based_source_separation/dataset/all_10_2/test.json'

# JSONファイルの読み込み
train_data = load_json_file(train_json_path)
test_data = load_json_file(test_json_path)

# パスの抽出と加工
train_paths = extract_and_process_paths(train_data)
test_paths = extract_and_process_paths(test_data)
print(train_data)
# 重複の確認
duplicates = find_duplicates(train_paths, test_paths)

# 結果の表示
print(duplicates)


In [41]:
!pip install mir_eval
!apt install -y python3-pyaudio
!pip install pyaudio

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3-pyaudio is already the newest version (0.2.11-1.1build1).
0 upgraded, 0 newly installed, 0 to remove and 4 not upgraded.


In [33]:
cd /home/code/code/DNN-based_source_separation-main/egs/tutorials/conv-tasnet

/home/code/code/DNN-based_source_separation-main/egs/tutorials/conv-tasnet


In [22]:
!chmod +x ./local/train.py

In [34]:
!. ./train.sh \
--epochs 101 \
--batch_size 2 \
--sample_rate 22000 \
--n_sources 2 \
--alpha 1 \
--N 32
#--continue_from "/home/code/code/DNN-based_source_separation-main/egs/tutorials/conv-tasnet/exp/2mix/trainable-trainable/sisdr/N64_L16_B128_H256_Sc128_P3_X6_R3/enc-relu_dilated1_separable1_causal0_prelu_norm1_mask-sigmoid/b2_e101_adam-lr1e-3-decay0_clip5/seed111/model/last.pth"

Namespace(alpha=1.0, batch_size=2, causal=0, continue_from='', criterion='sisdr', dec_basis='trainable', dilated=1, enc_basis='trainable', enc_nonlinear='relu', enc_onesided=0, enc_return_complex=0, epochs=101, kernel_size=16, loss_dir='./exp/2mix/trainable-trainable/sisdr/N32_L16_B128_H256_Sc128_P3_X6_R3/enc-relu_dilated1_separable1_causal0_prelu_norm1_mask-sigmoid/b2_e101_adam-lr1e-3-decay0_clip5/seed111/loss', lr=0.001, mask_nonlinear='sigmoid', max_norm=5.0, model_dir='./exp/2mix/trainable-trainable/sisdr/N32_L16_B128_H256_Sc128_P3_X6_R3/enc-relu_dilated1_separable1_causal0_prelu_norm1_mask-sigmoid/b2_e101_adam-lr1e-3-decay0_clip5/seed111/model', n_basis=32, n_sources=2, optimizer='adam', overwrite=0, sample_dir='./exp/2mix/trainable-trainable/sisdr/N32_L16_B128_H256_Sc128_P3_X6_R3/enc-relu_dilated1_separable1_causal0_prelu_norm1_mask-sigmoid/b2_e101_adam-lr1e-3-decay0_clip5/seed111/sample', sample_rate=22000, seed=111, sep_bottleneck_channels=128, sep_hidden_channels=256, sep_kern

In [ ]:
!. ./train.sh \
--epochs 100 \
--batch_size 2 \
--sample_rate 22000 \
--n_sources 2 \
--alpha 0.0

In [ ]:
!. ./train.sh \
--epochs 100 \
--batch_size 2 \
--sample_rate 22000 \
--n_sources 3 \
--alpha 0.0 \
--wav_root "../../../dataset/all_10_3" \
--train_json_path "../../../dataset/all_10_3/train.json" \
--valid_json_path "../../../dataset/all_10_3/test.json" 

In [ ]:
!. ./train.sh \
--epochs 105 \
--batch_size 2 \
--sample_rate 22000 \
--n_sources 3 \
--alpha 0.5 \
--wav_root "../../../dataset/all_10_3" \
--train_json_path "../../../dataset/all_10_3/train.json" \
--valid_json_path "../../../dataset/all_10_3/test.json"  \
--N 32

Namespace(alpha=0.5, batch_size=2, causal=0, continue_from='', criterion='sisdr', dec_basis='trainable', dilated=1, enc_basis='trainable', enc_nonlinear='relu', enc_onesided=0, enc_return_complex=0, epochs=105, kernel_size=16, loss_dir='./exp/3mix/trainable-trainable/sisdr/N64_L16_B128_H256_Sc128_P3_X6_R3/enc-relu_dilated1_separable1_causal0_prelu_norm1_mask-sigmoid/b2_e105_adam-lr1e-3-decay0_clip5/seed111/loss', lr=0.001, mask_nonlinear='sigmoid', max_norm=5.0, model_dir='./exp/3mix/trainable-trainable/sisdr/N64_L16_B128_H256_Sc128_P3_X6_R3/enc-relu_dilated1_separable1_causal0_prelu_norm1_mask-sigmoid/b2_e105_adam-lr1e-3-decay0_clip5/seed111/model', n_basis=64, n_sources=3, optimizer='adam', overwrite=0, sample_dir='./exp/3mix/trainable-trainable/sisdr/N64_L16_B128_H256_Sc128_P3_X6_R3/enc-relu_dilated1_separable1_causal0_prelu_norm1_mask-sigmoid/b2_e105_adam-lr1e-3-decay0_clip5/seed111/sample', sample_rate=22000, seed=111, sep_bottleneck_channels=128, sep_hidden_channels=256, sep_kern

In [ ]:
cd /home/code/code/DNN-based_source_separation-main/egs/tutorials/dprnn-tasnet

/home/code/code/DNN-based_source_separation-main/egs/tutorials/dprnn-tasnet


In [ ]:
!. ./train.sh \
--epochs 105 \
--batch_size 2 \
--sample_rate 22000 \
--n_sources 3 \
--alpha 0.5 \
--wav_root "../../../dataset/all_10_3" \
--train_json_path "../../../dataset/all_10_3/train.json" \
--valid_json_path "../../../dataset/all_10_3/test.json"  \
--N 32

Namespace(alpha=0.5, batch_size=2, causal=0, continue_from='', criterion='sisdr', dec_basis='trainable', enc_basis='trainable', enc_nonlinear='relu', enc_onesided=0, enc_return_complex=0, epochs=105, kernel_size=16, loss_dir='./exp/3mix/trainable-trainable/sisdr/N64_L16_F64_H128_K100_P50_B3/enc-relu_causal0_norm1_mask-sigmoid/b2_e105_adam-lr1e-3-decay0_clip0/seed111/loss', lr=0.001, mask_nonlinear='sigmoid', max_norm=0.0, model_dir='./exp/3mix/trainable-trainable/sisdr/N64_L16_F64_H128_K100_P50_B3/enc-relu_causal0_norm1_mask-sigmoid/b2_e105_adam-lr1e-3-decay0_clip0/seed111/model', n_basis=64, n_sources=3, optimizer='adam', overwrite=0, sample_dir='./exp/3mix/trainable-trainable/sisdr/N64_L16_F64_H128_K100_P50_B3/enc-relu_causal0_norm1_mask-sigmoid/b2_e105_adam-lr1e-3-decay0_clip0/seed111/sample', sample_rate=22000, seed=111, sep_bottleneck_channels=64, sep_chunk_size=100, sep_hidden_channels=128, sep_hop_size=50, sep_norm=1, sep_num_blocks=3, stride=None, train_json_path='../../../data

In [ ]:
!. ./train.sh \
--epochs 100 \
--batch_size 2 \
--sample_rate 22000 \
--n_sources 2 \
--alpha 0.0

In [ ]:
!. ./train.sh \
--epochs 101 \
--batch_size 2 \
--sample_rate 22000 \
--n_sources 2 \
--alpha 1  \
--N 32

In [ ]:
cd /home/code/code/DNN-based_source_separation-main/egs/tutorials/conv-tasnet

In [ ]:
!. ./train.sh \
--epochs 101 \
--batch_size 2 \
--sample_rate 22000 \
--n_sources 2 \
--alpha 1 \
--N 128
#--continue_from "/home/code/code/DNN-based_source_separation-main/egs/tutorials/conv-tasnet/exp/2mix/trainable-trainable/sisdr/N64_L16_B128_H256_Sc128_P3_X6_R3/enc-relu_dilated1_separable1_causal0_prelu_norm1_mask-sigmoid/b2_e101_adam-lr1e-3-decay0_clip5/seed111/model/last.pth"

In [ ]:
!. ./train.sh \
--epochs 105 \
--batch_size 2 \
--sample_rate 22000 \
--n_sources 3 \
--alpha 0.5 \
--wav_root "../../../dataset/all_10_3" \
--train_json_path "../../../dataset/all_10_3/train.json" \
--valid_json_path "../../../dataset/all_10_3/test.json"  \
--N 128

In [ ]:
cd /home/code/code/DNN-based_source_separation-main/egs/tutorials/dprnn-tasnet

In [ ]:
!. ./train.sh \
--epochs 101 \
--batch_size 2 \
--sample_rate 22000 \
--n_sources 2 \
--alpha 1 \
--N 128


In [ ]:
!. ./train.sh \
--epochs 105 \
--batch_size 2 \
--sample_rate 22000 \
--n_sources 3 \
--alpha 0.5 \
--wav_root "../../../dataset/all_10_3" \
--train_json_path "../../../dataset/all_10_3/train.json" \
--valid_json_path "../../../dataset/all_10_3/test.json"  \
--N 128